In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

from sklearn.metrics import r2_score
from scipy import stats

from sklearn.metrics import mean_squared_error

'''
Author: Enrico Ceccolini
    TODO
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
#interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
#end_time = pd.to_datetime('2014-11-01')

infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_short_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_short_jobs_real_pow"

In [2]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))

### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(long_jobs, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 35284 long jobs in the Andrea's interval
after the clean, jobs_to_nodes_whole_data contains 444610 records
merged_jobs_to_nodes contains 41216 records


In [3]:
short_jobs = pd.read_csv(infile_short_jobs + ".csv", index_col=0)
print("There are {} short jobs in the {}'s interval".format(short_jobs.shape[0], interval_comment))

There are 53253 short jobs in the Andrea's interval


In [4]:
merged_jobs_to_nodes.head()

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,n_cores_2_2,n_cores_3_1,user_short,pred_user,pred_userJob,node_id,ncpus,ngpus,nmics,mem_requested
0,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,...,16.0,0.0,alupi000,225.932207,198.527435,17,16,0,0,14680064
1,498459.node129,CPMD,parallel,2014-03-31 00:36:51,2014-03-31 00:36:51,2014-03-31 00:40:13,framondo@node129.eurora.cineca.it,1,16,14,...,0.0,0.0,framondo,138.451208,163.188808,9,16,0,0,14680064
2,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0@node129.eurora.cineca.it,6,96,84,...,16.0,0.0,vcalzia0,936.382877,1172.141601,9,16,0,0,14680064
3,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0@node129.eurora.cineca.it,6,96,84,...,16.0,0.0,vcalzia0,936.382877,1172.141601,28,16,0,0,14680064
4,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0@node129.eurora.cineca.it,6,96,84,...,16.0,0.0,vcalzia0,936.382877,1172.141601,25,16,0,0,14680064


## Calculate the mean core consumption

In [6]:
good_jobs = long_jobs[long_jobs['real_pow_quality'] > 0.5]

Separo i nodi che hanno eseguito esclusivamente su un tipo di nodo

In [7]:
long_jobs_2_1 = good_jobs[good_jobs['node_req'] == good_jobs['n_2_1']]
print("There are {} jobs completely running on this type of cpu".format(long_jobs_2_1.shape[0]))
long_jobs_2_2 = good_jobs[good_jobs['node_req'] == good_jobs['n_2_2']]
print("There are {} jobs completely running on this type of cpu".format(long_jobs_2_2.shape[0]))
long_jobs_3_1 = good_jobs[good_jobs['node_req'] == good_jobs['n_3_1']]
print("There are {} jobs completely running on this type of cpu".format(long_jobs_3_1.shape[0]))

There are 4786 jobs completely running on this type of cpu
There are 1426 jobs completely running on this type of cpu
There are 25315 jobs completely running on this type of cpu


In [8]:
mean_core_2_1 = long_jobs_2_1['real_pow'].sum() / long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
mean_core_2_2 = long_jobs_2_2['real_pow'].sum() / long_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_2))
mean_core_3_1 = long_jobs_3_1['real_pow'].sum() / long_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_3_1))

The average per core is 12.223003952684024 W
The average per core is 15.287422699162688 W
The average per core is 20.872660829336393 W


In [9]:
### Esclude the jobs with 0 quality of approximation
zero_quality_long_jobs_2_1 = long_jobs_2_1[long_jobs_2_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(zero_quality_long_jobs_2_1.shape[0]))
zero_quality_long_jobs_2_2 = long_jobs_2_2[long_jobs_2_2['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(zero_quality_long_jobs_2_2.shape[0]))
zero_quality_long_jobs_3_1 = long_jobs_3_1[long_jobs_3_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(zero_quality_long_jobs_3_1.shape[0]))

There are 4786 jobs completely running on this type of cpu with that quality
There are 1426 jobs completely running on this type of cpu with that quality
There are 25315 jobs completely running on this type of cpu with that quality


In [10]:
zero_quality_mean_core_2_1 = zero_quality_long_jobs_2_1['real_pow'].sum() / zero_quality_long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(zero_quality_mean_core_2_1))
zero_quality_mean_core_2_2 = zero_quality_long_jobs_2_2['real_pow'].sum() / zero_quality_long_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(zero_quality_mean_core_2_2))
zero_quality_mean_core_3_1 = zero_quality_long_jobs_3_1['real_pow'].sum() / zero_quality_long_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(zero_quality_mean_core_3_1))

The average per core is 12.223003952684024 W
The average per core is 15.287422699162688 W
The average per core is 20.872660829336393 W


In [18]:
### Take only jobs with a good quality of approximation
quality_long_jobs_2_1 = long_jobs_2_1[long_jobs_2_1['real_pow_quality'] >= 0.9]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_2_1.shape[0]))
quality_long_jobs_2_2 = long_jobs_2_2[long_jobs_2_2['real_pow_quality'] >= 0.9]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_2_2.shape[0]))
quality_long_jobs_3_1 = long_jobs_3_1[long_jobs_3_1['real_pow_quality'] >= 0.9]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_3_1.shape[0]))

There are 4716 jobs completely running on this type of cpu with that quality
There are 1397 jobs completely running on this type of cpu with that quality
There are 24884 jobs completely running on this type of cpu with that quality


In [19]:
quality_mean_core_2_1 = quality_long_jobs_2_1['real_pow'].sum() / quality_long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(quality_mean_core_2_1))
quality_mean_core_2_2 = zero_quality_long_jobs_2_2['real_pow'].sum() / quality_long_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(quality_mean_core_2_2))
quality_mean_core_3_1 = zero_quality_long_jobs_3_1['real_pow'].sum() / quality_long_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(quality_mean_core_3_1))

The average per core is 12.199523757603714 W
The average per core is 15.501496987371722 W
The average per core is 21.621871784108887 W


# Predict

In [4]:
infile_mean_core_consumptions = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_consumption_overAll"
mean_core_consumptions = pd.read_csv(infile_mean_core_consumptions + ".csv")

In [5]:
mean_core_consumptions

,job_quality,2_1_ghz,2_2_ghz,3_1_ghz
0,all,11.805211,15.155948,20.042817
1,medium,12.223004,15.501497,20.872661
2,high,12.199524,15.711896,21.621872


In [23]:
long_jobs['pred_overAll'] = 0.0
mean_core_consumptions_2_1_all = mean_core_consumptions[mean_core_consumptions['job_quality'] == 'medium']['2_1_ghz']
mean_core_consumptions_2_2_all = mean_core_consumptions[mean_core_consumptions['job_quality'] == 'medium']['2_2_ghz']
mean_core_consumptions_3_1_all = mean_core_consumptions[mean_core_consumptions['job_quality'] == 'medium']['3_1_ghz']

In [24]:
i = 0
for index, job in long_jobs.iterrows():
    job_n_cores_2_1 = job['n_cores_2_1']
    job_n_cores_2_2 = job['n_cores_2_2']
    job_n_cores_3_1 = job['n_cores_3_1']
    
    job_predicted_consumption = job_n_cores_2_1 * mean_core_consumptions_2_1_all + job_n_cores_2_2 * mean_core_consumptions_2_2_all + job_n_cores_3_1 * mean_core_consumptions_3_1_all
    
    long_jobs.at[index, 'pred_overAll'] = job_predicted_consumption
    
    if (i % 10000 == 0):
        print(i)
    i += 1

0
10000
20000
30000


In [25]:
# write to long_jobs
long_jobs.to_csv(infile_long_jobs + ".csv")

# Validate

In [3]:
good_quality_jobs = long_jobs[long_jobs['real_pow_quality'] != 0]
#print(math.sqrt(np.var(good_quality_jobs['real_pow'])))
#good_quality_jobs = long_jobs

mean_real_pow = long_jobs['real_pow'].mean()
print("the mean is {}".format(mean_real_pow))


good_quality_jobs['numerator'] = (good_quality_jobs['real_pow'] - good_quality_jobs['pred_overAll'])**2
good_quality_jobs['denumerator'] = (good_quality_jobs['real_pow'] - mean_real_pow)**2
N = good_quality_jobs['numerator'].shape[0]

r_2 = 1 - (good_quality_jobs['numerator'].sum() / good_quality_jobs['denumerator'].sum())
print("r_2 is {}".format(r_2))

rmse = math.sqrt(good_quality_jobs['numerator'].sum() / N)
print("RMSE is {}".format(rmse))
print("NRMSE is {}".format(rmse/mean_real_pow))

the mean is 110.35204538702968
r_2 is 0.5133103109341102
RMSE is 188.15289712851822
NRMSE is 1.705024102350104


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


### NRMSE

$$ NRMSE = \frac{\sqrt{(\sum^N_{i=1}{(P_i - P_i^*)^2})/N}}{\overline{P}} $$

### debug

In [18]:
long_jobs = long_jobs[long_jobs['real_pow_quality'] > 0.5]
print(long_jobs.shape[0])

32517


In [17]:
slope, intercept, r_value, p_value, std_err = stats.linregress(long_jobs['real_pow'], long_jobs['pred_overAll'])
print("R-squared: {}".format(r_value**2))
from sklearn.metrics import mean_squared_error
print("mean_quared_error: {}".format(mean_squared_error(long_jobs['real_pow'], long_jobs['pred_overAll'])))

R-squared: 0.5918227460516069
mean_quared_error: 72473.61063968857


In [83]:
slope, intercept, r_value, p_value, std_err = stats.linregress(long_jobs['real_pow'], long_jobs['pred_overAll_high'])
print("R-squared: {}".format(r_value**2))

R-squared: 0.5905750563103266
